In [ ]:
# Doing some Object Detection on a video
reader = imageio.get_reader('funny_dog.mp4') # We open the video.
fps = reader.get_meta_data()['fps'] # We get the fps frequence (frames per second).
writer = imageio.get_writer('output.mp4', fps = fps) # We create an output video with this same fps frequence.
for i, frame in enumerate(reader): # We iterate on the frames of the output video:
    frame = detect(frame, net.eval(), transform) # We call our detect function (defined above) to detect the object on the frame.
    writer.append_data(frame) # We add the next frame in the output video.
    print(i) # We print the number of the processed frame.
writer.close() # We close the process that handles the creation of the output video.

In [ ]:

import os
import sys
import random
import math
import numpy as np
import scipy.misc
import matplotlib
import matplotlib.pyplot as plt

import coco
import utils
import model as modellib
import visualize

%matplotlib inline 

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Path to trained weights file
# Download this file and place in the root of your 
# project (See README file for details)
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

In [ ]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.print()

In [ ]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

In [ ]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

In [ ]:
# Load a random image from the images folder
file_names = next(os.walk(IMAGE_DIR))[2]
x = len(file_names)
for k in range(x):
    image = scipy.misc.imread(os.path.join(IMAGE_DIR, file_names[k]))

    # Run detection
    results = model.detect([image], verbose=1)

    # Visualize results
    r = results[0]
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'])

In [ ]:
import imageio
import cv2

reader = imageio.get_reader('0002-20170519-2.mp4') # We open the video.
fps = reader.get_meta_data()['fps'] # We get the fps frequence (frames per second).
writer = imageio.get_writer('output.mp4', fps = fps)

for i, frame in enumerate(reader): # We iterate on the frames of the output video:
    #frame = scipy.misc.imread(frame) # We call our detect function (defined above) to detect the object on the frame.
    height, width = frame.shape[:2]
    print(height, width, "height, width")
    
    results = model.detect([frame], verbose=1)
    r = results[0]
    print(r['rois'][0], "rois 1",len(r['scores']))
    print(class_names[r['class_ids'][1]])
    viz = visualize.display_instances(frame, r['rois'], r['masks'], r['class_ids'],class_names, r['scores'])
    
    #как писать на изображение, только это в торче, а тут тф
    for j in range(len(r['scores'])): 
        if r['scores'][j] >= 0.6: # We take into account all the occurrences j of the class i that have a matching score larger than 0.6.
            #pt = (detections[0, i, j, 1:] * scale).numpy() # We get the coordinates of the points at the upper left and the lower right of the detector rectangle.
            #frame = frame[...,::-1]
            frame=cv2.rectangle(frame, (int(r['scores'][0]), int(r['scores'][1])), (int(r['scores'][2]), int(r['scores'][3])), (255, 0, 0), 4) # We draw a rectangle around the detected object.
            frame=cv2.putText(frame,class_names[r['class_ids'][j]], (int(r['scores'][0]), int(r['scores'][1])), cv2.FONT_HERSHEY_SIMPLEX, 4, (255, 255, 255), 2, cv2.LINE_AA) # We put the label of the class right above the rectangle.
            if (i==0):
                print(type(frame))
                plt.imshow(frame)
    
    writer.append_data(frame) # We add the next frame in the output video.
    
    print(i) # We print the number of the processed frame.
    if i==5:
        break
writer.close() # We close the process that handles the creation of the output video

In [ ]:
#http://www.sroboto.com/2017/09/pass-video-into-tensorflow-object.html

#https://github.com/smallcorgi/3D-Deepbox

#https://github.com/MagaliDrumare/How-to-make-object-recognition-in-a-video-with-PyTorch-SSD/blob/master/object_detection_commented.py
#https://dzone.com/articles/image-data-analysis-using-numpy-amp-opencv

#https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_image_display/py_image_display.html

#http://www.cvlibs.net/datasets/kitti/